In [5]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import pandas as pd

import sys
sys.path.insert(0, "/home/aisummer/mikhail_workspace/nlp-service")

from src.DocumentsParser.utils import DOCS_DIR_INFO_FILE

MAIN_PAGE_URL = "https://www.gost.ru/portal/gost/home/standarts/InformationSecurity"
SAVE_DIR = '../../data/infsec_gosts/docs'

In [33]:
def download_doc(url, save_path, save_name):
    response = requests.get(url, stream=True)
    with open(f"{save_path}/{save_name}", "wb") as handle:
        for data in response.iter_content():
            handle.write(data)

# извлекаем названия документов
# и ссылки на скачивание
def get_docs_info(html_page):
    info = []

    soup = BeautifulSoup(html_page)
    table = soup.select_one(".wcm-editor__content > table:nth-child(2)")
    for row in table.find_all("tr")[1:]:
        row_items = row.find_all("td")
        doc_title = row_items[1].text.strip()
        doc_link = row_items[-1].find_all("a", href=True)[0]['href']
        info.append([doc_title, doc_link])
    
    return info

def get_html_page(url):
    return requests.get(url).content.decode('utf-8')

In [34]:
html_page = get_html_page(MAIN_PAGE_URL)
info = get_docs_info(html_page)

for i in tqdm(range(len(info))):
    file_name = f"doc{i}.pdf"
    download_doc(info[i][1], SAVE_DIR, file_name)
    info[i].append(file_name)

100%|██████████| 89/89 [02:59<00:00,  2.02s/it]


In [35]:
df = pd.DataFrame(info, columns=["title", 'url', "filename"])
df.to_csv(f"{SAVE_DIR}/{DOCS_DIR_INFO_FILE}", sep=';', index=False)